In [1]:
from pysat.formula import CNF,WCNF
from pysat.card import *
from pysat.formula import IDPool
import itertools as it
def encode_problem(problem,enc_type=EncType.cardnetwrk,weighted = False):
    vpool = IDPool()
    var = lambda i: vpool.id(f'var_{i}')
    ch = lambda c,h: var(f"ch_{c}_{h}")
    cd = lambda c,d: var(f"cd_{c}_{d}")
    cr = lambda c,r: var(f"cr_{c}_{r}")
    kh = lambda k,h: var(f"kh_{k}_{h}")

    days = list(range(problem["days"]))
    hours = []
    def hour_of(d,h):
        return f"d{d}h{h}"
    def hours_of(d):
        return [hour_of(d,h) for h in range(problem["periods_per_day"])]
    for d in days:
        hours += hours_of(d)
    def day_of(hour):
        return hour.split("h")[0][1:]
    if weighted:
        cnf = WCNF()
    else:
        cnf = CNF()
    # Constraints
    # Relation between ch and cd:
    """If some course occurs in hour h, it also occurs in the day corresponding to h.
        So, for each course c and hour h, the following two-literal clause is needed:"""
    for course in problem["course_data"]:
        c = course["course_id"]
        for h in hours:
            cnf.append([-ch(c,h),cd(c,day_of(h))])
    """If some course occurs in a day d, it must also occur in some of the hours of d.
        So, for each course c and day d consisting of hours h1, h2, . . . hn, the following
        clause is needed:"""
    for course in problem["course_data"]:
        c = course["course_id"]
        for d in days:
            clause = [-cd(c,d)]
            for h in hours_of(d):
                clause.append(ch(c,h))
            cnf.append(clause)
    # Relation between ch and kh
    """If some course c occurs in hour h, all the curricula k1, k2, . . . , knk, to which c
        belongs occur in h. So, for each course c, hour h and curricula k1, k2, . . . , kn
        that include c, the following clauses are needed:"""
    for curriculum in problem["curricula_data"]:
        k = curriculum["curriculum_id"]
        for h in hours:
            for c in curriculum["members"]:
                cnf.append([-ch(c,h),kh(k,h)])
    """If some curriculum k occurs in hour h, then at least one of the courses belonging
        to k must also occur in h. So, for every hour h and curriculum k consisting of
        courses c1, c2, . . . , cn, the following clauses are needed:"""
    for curriculum in problem["curricula_data"]:
        k = curriculum["curriculum_id"]
        for h in hours:
            clause = [-kh(k,h)]
            for c in curriculum["members"]:
                clause.append(ch(c,h))
            cnf.append(clause)
    # Curriculum clashes
    """No two courses c and c' belonging to the same curriculum may
        be scheduled at the same hour. So, for each hour h and for each pair of distinct
        courses c, c0 belonging to the same curriculum, we have:"""
    for curriculum in problem["curricula_data"]:
        for h in hours:
            for c,c_p in it.combinations(curriculum["members"], 2):
                cnf.append([-ch(c,h),-ch(c_p,h)])
    # Teacher clashes
    """No two courses c and c' with the same teacher may be scheduled to
        the same hour. So, for each hour h and for each pair of distinct courses c, c'
        such that teacher(c) = teacher(c'), we have:"""
    for h in hours:
        for course,course_p in it.combinations(problem["course_data"], 2):
            if course["teacher"] == course_p["teacher"]:
                cnf.append([-ch(course["course_id"],h),-ch(course_p["course_id"],h)])
    # Room clashes
    """ No two courses c and c' may be scheduled to the same room at the
        same hour. So, for each room r, hour h, and pair of distinct courses c, c', we need:"""
    for room in problem["room_data"]:
        r = room["room_id"]
        for h in hours:
            for course,course_p in it.combinations(problem["course_data"], 2):
                c = course["course_id"]
                c_p = course_p["course_id"]
                cnf.append([-ch(c,h),-ch(c_p,h),-cr(c,r),-cr(c_p,r)])
    # Hour availability
    """For each course c with forbidden hours h1, h2, . . . , hn, we have the
        following one-literal clauses:"""
    for constraint in problem["constraints_data"]:
        c = constraint["course_id"]
        h = hour_of(constraint["day"],constraint["day_period"])
        cnf.append([-ch(c,h)])
    # Number of lectures
    """ For each course c exactly hours(c) of the set chc,h1, chc,h2, . . . , chc,hn must be true"""
    for course in problem["course_data"]:
        c = course["course_id"]
        #continue
        lits = [ch(c,h) for h in hours]
        bound = course["lectures"]
        clauses = CardEnc.equals(lits, bound=bound, encoding=enc_type,vpool=vpool)
        cnf.extend(clauses)
    # Room capacity
    """Each course must be scheduled to a room in which it fits. So, for each
        course c with number of students ns and for every room r with capacity cr such
        that cr < ns, we have:"""
    for course in problem["course_data"]:
        c = course["course_id"]
        for room in problem["room_data"]:
            r = room["room_id"]
            if room["capacity"] < course["students"]:
                cnf.append([-cr(c,r)])
    # Min working days
    """ For each course c, at least working days(c) literals of the set cdc,d1, cdc,d2, . . . , cdc,d5 should be true:"""
    for course in problem["course_data"]:
        c = course["course_id"]
        lits = [cd(c,d) for d in days]
        bound = course["min_working_days"]
        clauses = CardEnc.atleast(lits, bound=bound, encoding=enc_type,vpool=vpool)
        cnf.extend(clauses)
    # Isolated lectures
    """If some curriculum k occurs in hour h, then k must also occur in an
        hour before or after in the same day. For each curriculum k:"""
    for d in days:
        hs = hours_of(d)
        # first hour
        for curriculum in problem["curricula_data"]:
            k = curriculum["curriculum_id"]
            if weighted:
                cnf.append([-kh(k,hs[0]),kh(k,hs[1])],weight=2)
            else:
                cnf.append([-kh(k,hs[0]),kh(k,hs[1])])
        # last hour
        for curriculum in problem["curricula_data"]:
            k = curriculum["curriculum_id"]
            if weighted:
                cnf.append([-kh(k,hs[-1]),kh(k,hs[-2])],weight=2)
            else:
                cnf.append([-kh(k,hs[-1]),kh(k,hs[-2])])
        for curriculum in problem["curricula_data"]:
            k = curriculum["curriculum_id"]
            for idx in range(len(hs)):
                if idx==0 or idx==len(hs)-1:
                    continue
                if weighted:
                    cnf.append([-kh(k,hs[idx]),kh(k,hs[idx-1]),kh(k,hs[idx+1])],weight=2)
                else:
                    cnf.append([-kh(k,hs[idx]),kh(k,hs[idx-1]),kh(k,hs[idx+1])])
    # Room stability
    """ Each course must be scheduled to exactly one room. So, if there are
        rooms r1, . . . , rn, for each course c we have"""
    for course in problem["course_data"]:
        c = course["course_id"]
        lits = [cr(c,room["room_id"]) for room in problem["room_data"]]
        clauses = CardEnc.equals(lits, bound=1, encoding=EncType.pairwise,vpool=vpool)
        cnf.extend(clauses)
    return cnf,vpool


In [2]:
def check(problem,vpool,model,weighted = False):
    ch_set = set()
    cd_set = set()
    cr_set = set()
    kh_set = set()
    for lit in model:
        obj = vpool.obj(lit)
        if lit>0 and obj is not None:
            obj = obj[4:]
            if obj.startswith("ch"):
                c,h = obj[3:].split("_")
                ch_set.add((c,h))
            elif obj.startswith("cd"):
                c,d = obj[3:].split("_")
                cd_set.add((c,d))
            elif obj.startswith("cr"):
                c,r = obj[3:].split("_")
                cr_set.add((c,r))
            elif obj.startswith("kh"):
                k,h = obj[3:].split("_")
                kh_set.add((k,h))
    days = list(range(problem["days"]))
    hours = []
    def hour_of(d,h):
        return f"d{d}h{h}"
    def hours_of(d):
        return [hour_of(d,h) for h in range(problem["periods_per_day"])]
    for d in days:
        hours += hours_of(d)
    def day_of(hour):
        return hour.split("h")[0][1:]
    curriculums = {}
    for curriculum in problem["curricula_data"]:
        curriculums[curriculum["curriculum_id"]] = curriculum
    courses = {}
    for course in problem["course_data"]:
        courses[course["course_id"]] = course
    rooms = {}
    for room in problem["room_data"]:
        rooms[room["room_id"]] = room
    # Relation between ch and cd:
    """If some course occurs in hour h, it also occurs in the day corresponding to h.
        So, for each course c and hour h, the following two-literal clause is needed:"""
    for c,h in ch_set:
        if (c,day_of(h)) not in cd_set:
            print(f"{problem["name"]}: Failed on ch_{c}_{h} -> cd_{c}_{day_of(h)}")
            return False
    """If some course occurs in a day d, it must also occur in some of the hours of d.
        So, for each course c and day d consisting of hours h1, h2, . . . hn, the following
        clause is needed:"""
    for c,d in cd_set:
        is_in_set = False
        for h in hours_of(d):
            if (c,h) in ch_set:
                is_in_set = True
                break
        if not is_in_set:
            print(f"{problem["name"]}: Failed on cd_{c}_{d}")
            return False
    # Relation between ch and kh
    """If some course c occurs in hour h, all the curricula k1, k2, . . . , knk, to which c
        belongs occur in h. So, for each course c, hour h and curricula k1, k2, . . . , kn
        that include c, the following clauses are needed:"""
    for c,h in ch_set:
        for k in curriculums:
            if c in curriculums[k]["members"] and (k,h) not in kh_set:
                print(f"{problem["name"]}: Failed on ch_{c}_{h} -> kh_{k}_{h}")
                return False
    """If some curriculum k occurs in hour h, then at least one of the courses belonging
        to k must also occur in h. So, for every hour h and curriculum k consisting of
        courses c1, c2, . . . , cn, the following clauses are needed:"""
    for k,h in kh_set:
        is_in_set = False
        for c in curriculums[k]["members"]:
            if (c,h) in ch_set:
                is_in_set = True
                break
        if not is_in_set:
            print(f"{problem["name"]}: Failed on kh_{k}_{h}")
            return False
    # Curriculum clashes
    """No two courses c and c' belonging to the same curriculum may
        be scheduled at the same hour. So, for each hour h and for each pair of distinct
        courses c, c0 belonging to the same curriculum, we have:"""
    for curriculum in problem["curricula_data"]:
        for c,c_p in it.combinations(curriculum["members"], 2):
            if (c,h) in ch_set and (c_p,h) in ch_set:
                print(f"{problem["name"]}: Failed on ch_{c}_{h} and ch_{c_p}_{h}")
                return False
    # Teacher clashes
    """No two courses c and c' with the same teacher may be scheduled to
        the same hour. So, for each hour h and for each pair of distinct courses c, c'
        such that teacher(c) = teacher(c'), we have:"""
    for h in hours:
        for course,course_p in it.combinations(problem["course_data"], 2):
            if course["teacher"] == course_p["teacher"]:
                if (course["course_id"],h) in ch_set and (course_p["course_id"],h) in ch_set:
                    print(f"{problem["name"]}: Failed on ch_{course['course_id']}_{h} and ch_{course_p['course_id']}_{h}")
                    return False
    # Room clashes
    """ No two courses c and c' may be scheduled to the same room at the
        same hour. So, for each room r, hour h, and pair of distinct courses c, c', we need:"""
    for r,h in cr_set:
        for course,course_p in it.combinations(problem["course_data"], 2):
            c = course["course_id"]
            c_p = course_p["course_id"]
            if (c,h) in ch_set and (c_p,h) in ch_set and (c,r) in cr_set and (c_p,r) in cr_set:
                print(f"{problem["name"]}: Failed on ch_{c}_{h} and ch_{c_p}_{h} and cr_{c}_{r} and cr_{c_p}_{r}")
                return False
    # Hour availability
    """For each course c with forbidden hours h1, h2, . . . , hn, we have the
        following one-literal clauses:"""
    for constraint in problem["constraints_data"]:
        c = constraint["course_id"]
        h = hour_of(constraint["day"],constraint["day_period"])
        if (c,h) in ch_set:
            print(f"{problem["name"]}: Failed on ch_{c}_{h}")
            return False
    # Number of lectures
    """ For each course c exactly hours(c) of the set chc,h1, chc,h2, . . . , chc,hn must be true"""
    for course in problem["course_data"]:
        c = course["course_id"]
        bound = course["lectures"]
        for c_p,h in ch_set:
            if c==c_p:
                bound -= 1
        if bound != 0:
            print(f"{problem["name"]}: Failed on course {c}")
            return False
    # Room capacity
    """Each course must be scheduled to a room in which it fits. So, for each
        course c with number of students ns and for every room r with capacity cr such
        that cr < ns, we have:"""
    for c,r in cr_set:
        if courses[c]["students"] > rooms[r]["capacity"]:
            print(f"{problem["name"]}: Failed on cr_{c}_{r}")
            return False
    # Min working days
    """ For each course c, at least working days(c) literals of the set cdc,d1, cdc,d2, . . . , cdc,d5 should be true:"""
    for course in problem["course_data"]:
        c = course["course_id"]
        bound = course["min_working_days"]
        for c_p,d in cd_set:
            if c==c_p:
                bound -= 1
        if bound > 0:
            print(f"{problem["name"]}: Failed on course {c}")
            return False
    if not weighted:
        # Isolated lectures
        """If some curriculum k occurs in hour h, then k must also occur in an
            hour before or after in the same day. For each curriculum k:"""
        for d in days:
            hs = hours_of(d)
            # first hour
            for k in curriculums:
                if (k,hs[0]) in kh_set and (k,hs[1]) not in kh_set:
                    print(f"{problem["name"]}: Failed on kh_{k}_{hs[0]} and kh_{k}_{hs[1]}")
                    return False
            # last hour
            for k in curriculums:
                if (k,hs[-1]) in kh_set and (k,hs[-2]) not in kh_set:
                    print(f"{problem["name"]}: Failed on kh_{k}_{hs[-1]} and kh_{k}_{hs[-2]}")
                    return False
            for k in curriculums:
                for idx in range(len(hs)):
                    if idx==0 or idx==len(hs)-1:
                        continue
                    if (k,hs[idx]) in kh_set and (k,hs[idx-1]) not in kh_set and (k,hs[idx+1]) not in kh_set:
                        print(f"{problem["name"]}: Failed on kh_{k}_{hs[idx]} and kh_{k}_{hs[idx-1]} and kh_{k}_{hs[idx+1]}")
                        return False
    # Room stability
    """ Each course must be scheduled to exactly one room. So, if there are
        rooms r1, . . . , rn, for each course c we have"""
    for course in problem["course_data"]:
        c = course["course_id"]
        bound = 1
        for c_p,r in cr_set:
            if c==c_p:
                bound -= 1
        if bound != 0:
            print(f"{problem["name"]}: Failed on course {c}")
            return False
    return True

In [80]:
from glob import glob
from pysat.solvers import *
from ctt_parser import CTTParser
for ctt_instance in glob("itc2007/*.ctt"):
    if ctt_instance == "itc2007/comp01.ctt":
        continue
    problem = CTTParser(ctt_instance).get_data()
    
    cnf, vpool = encode_problem(problem)
    with Solver(bootstrap_with=cnf,use_timer=True) as s:
        if s.solve():
            status = "SAT"
            model = s.get_model()
        else:
            status = "UNSAT"
        print(f"{ctt_instance} {status} {s.time()}")

itc2007/comp18.ctt UNSAT 0.011014216000006627
itc2007/comp08.ctt UNSAT 0.04526058900000862
itc2007/comp13.ctt UNSAT 0.04297391900001912
itc2007/comp03.ctt UNSAT 0.036627838000015345
itc2007/comp11.ctt SAT 0.6300423900000283
itc2007/comp05.ctt UNSAT 0.015394089999972493
itc2007/comp16.ctt UNSAT 1.2193000031857082e-05
itc2007/comp20.ctt UNSAT 0.12056545500001903
itc2007/comp17.ctt UNSAT 1.3835999993716541e-05
itc2007/comp12.ctt UNSAT 1.3304999981755827e-05
itc2007/comp04.ctt UNSAT 0.03985092200002782
itc2007/comp06.ctt UNSAT 1.4206999992438796e-05
itc2007/comp15.ctt UNSAT 0.03259582999999111
itc2007/comp21.ctt UNSAT 0.053529024999988906
itc2007/comp19.ctt UNSAT 1.3344999956643733e-05
itc2007/comp07.ctt UNSAT 0.1662791269999957
itc2007/comp10.ctt UNSAT 0.07762432200001967
itc2007/comp02.ctt UNSAT 0.0411585260000038
itc2007/comp14.ctt UNSAT 0.0474812919999863
itc2007/comp09.ctt UNSAT 0.03274758100002373


In [3]:
from glob import glob
from pysat.examples.rc2 import RC2
from ctt_parser import CTTParser
import concurrent.futures
import multiprocessing
import itertools as it

from pysat.solvers import Solver
def get_solver(cnf, weighted=False):
    # Create the solver based on whether it's weighted or not
    if weighted:
        return RC2(cnf)
    else:
        return Solver(bootstrap_with=cnf, use_timer=True)

def solve_ctt(ctt_instance, logfile, weighted=False):
    # Load problem data and encode
    problem = CTTParser(ctt_instance).get_data()
    cnf, vpool = encode_problem(problem, weighted=weighted)
    model = None
    result_status = None
    result_time = None
    
    # Create solver and process results
    with get_solver(cnf, weighted) as solver:
        result = solver.compute() if weighted else solver.solve()
        result_status = "SAT" if result else "UNSAT"
        result_time = solver.oracle_time() if weighted else solver.time()
        if result:
            model = solver.model if weighted else solver.get_model()
            if not check(problem, vpool, model, weighted):
                print(f"Failed on instance {ctt_instance}, weighted {weighted}.")
                result_status = "FAIL"

    # Write results to the logfile
    with open(logfile, "w") as f:
        f.write(f"{ctt_instance} {result_status} {result_time} {solver.cost if weighted else ''}\n")
        if model:
            for v in model:
                if vpool.obj(v) is not None and v > 0:
                    f.write(f"{vpool.obj(v)}\n")

def call_with_timeout(ctt_instance, weighted=False, time_limit=5):
    # Set up the log file
    logfile = ctt_instance.replace(".ctt", ".weighted.log" if weighted else ".log")

    # Define a wrapper function for timeout handling
    def solve_wrapper():
        solve_ctt(ctt_instance, logfile, weighted)

    # Run the function in a separate process
    process = multiprocessing.Process(target=solve_wrapper)
    process.start()
    process.join(timeout=time_limit)

    # Handle timeout by terminating the process and logging
    if process.is_alive():
        process.terminate()
        process.join()
        with open(logfile, 'w') as log:
            log.write(f"{ctt_instance} timeout {time_limit}\n")
        print(f"Timeout reached for instance {ctt_instance}, weighted {weighted}.")
        return False
    else:
        print(f"Completed instance {ctt_instance}, weighted {weighted} within time limit.")
        return True

# Run many instances in parallel with up to 8 concurrent processes
def run_many_in_parallel(instances, time_limit=600, max_workers=8):
    results = []
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks and maintain only 8 active at a time
        futures = {
            executor.submit(call_with_timeout, instance, weighted, time_limit): (instance, weighted)
            for instance, weighted in it.product(instances, [True, False])
        }
        
        # Collect results efficiently as tasks complete
        for future in concurrent.futures.as_completed(futures):
            instance, weighted = futures[future]
            try:
                result = future.result()
                results.append((instance, weighted, result))
            except Exception as exc:
                print(f"Instance {instance} generated an exception: {exc}")
                results.append((instance, weighted, None))
    return results

In [5]:
run_many_in_parallel(glob("itc2007/*.ctt"))

Completed instance itc2007/comp18.ctt, weighted False within time limit.
Completed instance itc2007/comp11.ctt, weighted True within time limit.
Completed instance itc2007/comp03.ctt, weighted False within time limit.
Completed instance itc2007/comp11.ctt, weighted False within time limit.
Completed instance itc2007/comp13.ctt, weighted False within time limit.
Completed instance itc2007/comp08.ctt, weighted False within time limit.
Completed instance itc2007/comp05.ctt, weighted True within time limit.
Completed instance itc2007/comp05.ctt, weighted False within time limit.
Completed instance itc2007/comp13.ctt, weighted True within time limit.
Completed instance itc2007/comp08.ctt, weighted True within time limit.
Completed instance itc2007/comp17.ctt, weighted False within time limit.
Completed instance itc2007/comp16.ctt, weighted False within time limit.
Completed instance itc2007/comp20.ctt, weighted False within time limit.
Completed instance itc2007/comp16.ctt, weighted True wi

[('itc2007/comp18.ctt', False, True),
 ('itc2007/comp11.ctt', True, True),
 ('itc2007/comp03.ctt', False, True),
 ('itc2007/comp11.ctt', False, True),
 ('itc2007/comp13.ctt', False, True),
 ('itc2007/comp08.ctt', False, True),
 ('itc2007/comp05.ctt', True, True),
 ('itc2007/comp05.ctt', False, True),
 ('itc2007/comp13.ctt', True, True),
 ('itc2007/comp08.ctt', True, True),
 ('itc2007/comp17.ctt', False, True),
 ('itc2007/comp16.ctt', False, True),
 ('itc2007/comp20.ctt', False, True),
 ('itc2007/comp16.ctt', True, True),
 ('itc2007/comp12.ctt', False, True),
 ('itc2007/comp04.ctt', False, True),
 ('itc2007/comp04.ctt', True, True),
 ('itc2007/comp06.ctt', False, True),
 ('itc2007/comp17.ctt', True, True),
 ('itc2007/comp06.ctt', True, True),
 ('itc2007/comp15.ctt', False, True),
 ('itc2007/comp20.ctt', True, True),
 ('itc2007/comp21.ctt', False, True),
 ('itc2007/comp19.ctt', True, True),
 ('itc2007/comp19.ctt', False, True),
 ('itc2007/comp18.ctt', True, False),
 ('itc2007/comp03.ctt'